# Project - YouTube Videos Summary Generator

### OpenAI Authentication

In [ ]:
import openai
import os

In [ ]:
openai.api_key = os.getenv('OPENAI_API_KEY')

### import custom package and setup path for it

In [ ]:
import sys
from pathlib import Path

# in jupyter (lab / notebook), based on notebook path

# print(f"Path.cwd(): {Path.cwd()}")
module_path = str(Path.cwd().parents[0])

if module_path not in sys.path:
    sys.path.append(module_path)

from common.usage import print_token_usage

In [ ]:
# !pip -q install pytube

In [ ]:
def youtube_audio_downloader(link):
    from pytube import YouTube
    import os
    import re
    if 'youtube.com' not in link:
        print('Invalid YouTube link!')
        return False
    
    yt = YouTube(link)
    
    audio = yt.streams.filter(only_audio=True).first()
    print('Downloading the audio stream ...', end='')
    output_file = audio.download()
    if os.path.exists(output_file):
        print('Done!')
    else:
        print('Error downloading the file!')
        return False
    
    basename = os.path.basename(output_file)
    name, extension = os.path.splitext(basename)
    audio_file = f'{name}.mp3'
    audio_file = re.sub('\s+', '-', audio_file)
    os.rename(basename, audio_file)
    return audio_file


In [ ]:
link = 'https://www.youtube.com/watch?v=G7KNmW9a75Y' # Miley Cyrus - Flowers

mp3_file = youtube_audio_downloader(link)
print(mp3_file)